In [82]:
import pandas as pd
import pickle

In [83]:
with open("../course_data/courses.pickle", "rb") as f:
    data_dict = pickle.load(f)

In [84]:
df = pd.DataFrame.from_dict(data_dict)
df

,lmt_yn,nemo_yn,eng100,year,tutorial_yn,mooc_yn,isu_nm,courgrad_cd,cour_cd,rowid,...,cour_nm,dept_cd,cour_cls,drop_lmt_yn,time,apply_dept,exch_cor_yn,no_supervisor_yn,col_cd,detailed_info
0,Y,N,N,2024,N,N,전공선택,0136,ACEE123,1,...,건축시스템의이해,5204,00,N,3(3),,Y,N,9999,"{'수업유형': '대면', '특별유형': [], '수업구성요소': [], '성적평가..."
1,Y,N,N,2024,N,N,전공선택,0136,ACEE219,2,...,환경화학(영강),5204,00,N,3(3),[*에코업track],N,N,9999,"{'수업유형': '대면', '특별유형': [], '수업구성요소': ['이론강의', ..."
2,Y,N,N,2024,N,N,전공선택,0136,ACEE227,3,...,유체역학및실험,5204,01,N,3(4),[*에코업track],N,N,9999,"{'수업유형': '대면', '특별유형': [], '수업구성요소': [], '성적평가..."
3,Y,N,N,2024,N,N,전공선택,0136,ACEE227,4,...,유체역학및실험(영강),5204,02,N,3(4),[*에코업track],N,N,9999,"{'수업유형': '대면', '특별유형': [], '수업구성요소': ['이론강의', ..."
4,Y,N,N,2024,N,N,전공선택,0136,ACEE228,5,...,구조역학I(영강),5204,00,N,3(4),[*에코업track],N,N,9999,"{'수업유형': '대면', '특별유형': [], '수업구성요소': [], '성적평가..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33429,Y,N,N,2020,N,N,전공필수,0136,TEEN303,3202,...,혁신성장창업트렌드,6544,00,N,3(3),,N,N,9999,"{'수업유형': None, '특별유형': [], '수업구성요소': [], '성적평가..."
33430,Y,N,N,2020,N,N,전공필수,0136,TEEN306,3203,...,기술창업전략,6544,00,N,3(3),,N,N,9999,"{'수업유형': None, '특별유형': [], '수업구성요소': ['이론강의', ..."
33431,Y,N,N,2020,N,N,전공필수,0136,TEEN312,3204,...,창업현장실습Ⅱ,6544,01,N,6(0),면담필수/인턴과정,Y,Y,9999,"{'수업유형': None, '특별유형': ['현장실습'], '수업구성요소': ['실..."
33432,Y,N,N,2020,N,N,전공필수,0136,TEEN312,3205,...,창업현장실습Ⅱ,6544,02,N,6(0),면담필수/(사업)대표과정,Y,Y,9999,"{'수업유형': None, '특별유형': ['현장실습'], '수업구성요소': ['실..."


해당 데이터셋에서 하나의 필드에 값이 하나로 결정돼있는 필드들이 있습니다. (waiting_yn, col_cd...) </br>
해당 필드들을 날려줍니다.

In [85]:
unique_counts = {}
for col in df.columns:
    if df[col].apply(lambda x: isinstance(x, dict)).any():
        continue
    n_unique = df[col].nunique()
    unique_counts[col] = n_unique
unique_one_cols = [col for col, count in unique_counts.items() if count == 1]
df.drop(columns=unique_one_cols, axis=1, inplace=True)

영향이 없거나 필요 없는 필드가 존재합니다 (rowid, flexible_to_dt...) </br>
또한 params는 학수번호와 분반으로 유도가 되는 필드값이므로 같이 날립니다.

In [86]:
df.drop(columns=["rowid", "params", "flexible_to_dt", "flexible_fr_dt", "cour_div"], axis=1, inplace=True)

Y/N 값을 불리언 값으로 변환합니다.

In [87]:
yn_columns = [col for col in df.columns if col.lower().endswith('yn')]
yn_columns += ["eng100"]
for yn_column in yn_columns:
    df[yn_column] = df[yn_column].map({'Y': True, 'N': False}).astype(bool)

|   | time |
|---|------|
| 0 | 3(3) |
| 1 | 3(3) |
| 2 | 3(4) |
| 3 | 3(4) |

time 필드의 경우 `학점(강의 시간)`으로 정의 되어 있어 둘을 각각의 필드로 구분돼야합니다. </br>
학점의 경우 credit 필드로 이미 정의 돼 있으므로 time 필드 내에서 강의 시간만 따로 추출하여 값을 대체합니다.

In [88]:
def get_time(value:str):
    s = value.find("(")
    e = value.find(")")
    return value[s + 1:e]
df["time"] = df["time"].apply(get_time).astype(float)

year 필드 Object형식을 숫자값으로 변환합니다

In [89]:
df['year'] = df['year'].astype(int)

In [90]:
df["isu_nm"].unique()

array(['전공선택', '전공필수', '학문의기초', '교양', '교직(비사대)', '군사학', '학부공통', '교직',
       '평생교육사'], dtype=object)

isu_nm 필드는 과목의 이수구분으로 순서가 없는 범주형 데이터이므로 </br>
one-hot 인코딩을 통해 `전공선택`, `전공필수`, `학문의기초`... 등 각 필드를 새로 만들어 불리언 값을 배정합니다.

In [91]:
df = pd.get_dummies(df, columns=['isu_nm'], drop_first=True, prefix='이수구분', prefix_sep='.')

In [92]:
df.time_room[::-1]

33433                          금(6-8) 서관 225호
33432                                        
33431                                        
33430                    화(5-6) 하나과학관 지하 216호
33429                         목(5-6) 과도관 408호
                         ...                 
4          화(2) 창의관 116호<br>\n목(2-3) 창의관 116호
3        화(3-4) 신공학관B105호<br>\n목(4) 신공학관B105호
2        화(3-4) 신공학관B104호<br>\n목(4) 신공학관B104호
1          월(4) 신공학관B104호<br>\n수(4) 신공학관B104호
0                             월(6-8) 창의관 205호
Name: time_room, Length: 33434, dtype: object

`time_room` 필드는 수업요일(교시) 장소 순으로 들어가 있으므로 유의미한 정보를 추출해내기 위해 one hot 인코딩을 진행합니다. </br>
ㅇ요일 ㅇ교시 수업 필드를 만들어 해당 교시에 수업이 있으면 장소를 없으면 `Nan`을 채워넣습니다</br>
|   | 월요일 1교시 | 월요일 2교시 | ... |
|---|------------|-------------|----|
| 0 | 교육관 313호      |   Nan       | ... |
| 1 | Nan     |   미정        | ... |


In [93]:
import re


def extract_partial_matches(text):
    pattern = r'([월화수목금토일])\((\d+)(?:-(\d+))?\)\s*(.*?)(?=<br>|$)'
    matches = re.findall(pattern, text)

    results = []
    for match in matches:
        week_of_day = match[0]
        start_period = int(match[1])
        end_period = int(match[2]) if match[2] else start_period
        room = match[3].strip()
        results.append((week_of_day, start_period, end_period, room))

    return results

all_columns = set()

# 각 row 처리
expanded_data = []
for schedule_entries in df["time_room"]:
    record = {}
    for day, start_period, end_period, room in extract_partial_matches(schedule_entries):
        for p in range(start_period, end_period + 1):
            room_key = f"{day}요일 {p}교시 수업"
            record[room_key] = room if room != '' else '미정'
            all_columns.add(room_key)
    expanded_data.append(record)

# DataFrame으로 변환하고 누락된 컬럼 보정
expanded_df = pd.DataFrame(expanded_data)
df = pd.concat([df, expanded_df], axis=1)
df.drop(columns=["time_room"], axis=1, inplace=True)
expanded_df

,월요일 6교시 수업,월요일 7교시 수업,월요일 8교시 수업,월요일 4교시 수업,수요일 4교시 수업,화요일 3교시 수업,화요일 4교시 수업,목요일 4교시 수업,화요일 2교시 수업,목요일 2교시 수업,...,화요일 11교시 수업,수요일 0교시 수업,화요일 0교시 수업,월요일 0교시 수업,화요일 12교시 수업,목요일 11교시 수업,목요일 12교시 수업,금요일 12교시 수업,목요일 0교시 수업,금요일 13교시 수업
0,창의관 205호,창의관 205호,창의관 205호,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,신공학관B104호,신공학관B104호,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,신공학관B104호,신공학관B104호,신공학관B104호,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,신공학관B105호,신공학관B105호,신공학관B105호,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,창의관 116호,창의관 116호,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33432,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
filed_name_dict = {
    'year': '년도',
    'term': '학기',
    'cour_cd': '학수번호',
    'cour_cls': '분반',
    'isu_nm': '이수구분',
    'department': '개설학과',
    'cour_nm': '교과목명',
    'prof_nm': '담당교수',
    'time_room': '강의교시',
    'apply_dept': '학과제한',
    'absolute_yn': '상대평가',
    'lmt_yn': '인원제한',
    'exch_cor_yn': '교환학생',
    'attend_free_yn': '출석확인자율화',
    'no_supervisor_yn': '무감독시험',
    'flexible_term': '유연학기',
    'mooc_yn': 'MOOC',
    'flipped_class_yn': 'Flipped',
    'nemo_yn': 'NeMo',
    'eng100': '영강',
    'drop_lmt_yn': '수강포기제한',
    'time' : '강의시간',
    'credit' : '학점',
    'tutorial_yn': "튜토리얼"
}
df.rename(columns=filed_name_dict, inplace=True)

In [95]:
df

,인원제한,NeMo,영강,년도,튜토리얼,MOOC,학수번호,학기,학점,개설학과,...,화요일 11교시 수업,수요일 0교시 수업,화요일 0교시 수업,월요일 0교시 수업,화요일 12교시 수업,목요일 11교시 수업,목요일 12교시 수업,금요일 12교시 수업,목요일 0교시 수업,금요일 13교시 수업
0,True,False,False,2024,False,False,ACEE123,1R,3.0,건축사회환경공학부,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,False,False,2024,False,False,ACEE219,1R,3.0,건축사회환경공학부,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,False,False,2024,False,False,ACEE227,1R,3.0,건축사회환경공학부,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,True,False,False,2024,False,False,ACEE227,1R,3.0,건축사회환경공학부,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,True,False,False,2024,False,False,ACEE228,1R,3.0,건축사회환경공학부,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33429,True,False,False,2020,False,False,TEEN303,2R,3.0,기술창업융합전공,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33430,True,False,False,2020,False,False,TEEN306,2R,3.0,기술창업융합전공,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33431,True,False,False,2020,False,False,TEEN312,2R,6.0,기술창업융합전공,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33432,True,False,False,2020,False,False,TEEN312,2R,6.0,기술창업융합전공,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
df["수업유형"] = df["detailed_info"].apply(lambda x: x.get("수업유형", None))
one_hot = pd.get_dummies(df["수업유형"], prefix="수업유형")
df.drop(columns=["수업유형"], axis=1)
df = pd.concat([df, one_hot], axis=1)

In [104]:
df

,인원제한,NeMo,영강,년도,튜토리얼,MOOC,학수번호,학기,학점,개설학과,...,수업유형_원격,수업유형_원격(녹화),수업유형_원격(실시간),수업유형_혼합,수업유형_대면,수업유형_병행,수업유형_원격,수업유형_원격(녹화),수업유형_원격(실시간),수업유형_혼합
0,True,False,False,2024,False,False,ACEE123,1R,3.0,건축사회환경공학부,...,False,False,False,False,True,False,False,False,False,False
1,True,False,False,2024,False,False,ACEE219,1R,3.0,건축사회환경공학부,...,False,False,False,False,True,False,False,False,False,False
2,True,False,False,2024,False,False,ACEE227,1R,3.0,건축사회환경공학부,...,False,False,False,False,True,False,False,False,False,False
3,True,False,False,2024,False,False,ACEE227,1R,3.0,건축사회환경공학부,...,False,False,False,False,True,False,False,False,False,False
4,True,False,False,2024,False,False,ACEE228,1R,3.0,건축사회환경공학부,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33429,True,False,False,2020,False,False,TEEN303,2R,3.0,기술창업융합전공,...,False,False,False,False,False,False,False,False,False,False
33430,True,False,False,2020,False,False,TEEN306,2R,3.0,기술창업융합전공,...,False,False,False,False,False,False,False,False,False,False
33431,True,False,False,2020,False,False,TEEN312,2R,6.0,기술창업융합전공,...,False,False,False,False,False,False,False,False,False,False
33432,True,False,False,2020,False,False,TEEN312,2R,6.0,기술창업융합전공,...,False,False,False,False,False,False,False,False,False,False


In [106]:
for a in df["detailed_info"][:10]:
    print(a["평가방법"])

{}
{}
{'출석': '10 %', '과제': '10 %', '중간고사': '40 %', '기말고사': '40 %', '합계': '100%'}
{}
{}
{}
{}
{}
{}
{}


In [107]:
df_expanded = pd.json_normalize(df["detailed_info"])

In [108]:
df_expanded

,수업유형,특별유형,수업구성요소,성적평가,평가방법.출석,평가방법.과제,평가방법.중간고사,평가방법.기말고사,평가방법.합계,평가방법.참여,...,평가방법.수업참여도및출석,평가방법.중간과제(팀별 사업계획 중간 발표),평가방법.기말과제 I(팀별 고객 리서치 상세 보고서 제출),평가방법.기말과제 II(팀별 프로토타입 디자인 및 개발),평가방법.기말과제 III(팀별 사업계획서 제출),평가방법.기말과제 IV(IR 발표),평가방법.기말발표및과제,평가방법.탐구발표 및 출석,평가방법.과제 및 탐구발표,평가방법.4 Homework
0,대면,[],[],[절대평가],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,대면,[],"[이론강의, 퀴즈, Q&A]",[절대평가],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,대면,[],[],[절대평가],10 %,10 %,40 %,40 %,100%,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,대면,[],"[이론강의, 실험]",[절대평가],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,대면,[],[],[절대평가],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33429,None,[],[],[절대평가],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33430,None,[],"[이론강의, 발표, 토론, 실습, 협동학습, 개별지도]",[절대평가],20 %,NaN,NaN,NaN,100%,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33431,None,[현장실습],[실습],"[절대평가, P/F]",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33432,None,[현장실습],[실습],"[절대평가, P/F]",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [136]:
# 특별유형 one-hot
special = df_expanded[['특별유형']].explode('특별유형')
special_dummies = pd.get_dummies(special['특별유형'], prefix='특별유형')
special_ohe = special_dummies.groupby(special.index).sum()
special_ohe = special_ohe.reindex(df_expanded.index, fill_value=0)

# 수업구성요소 one-hot
course = df_expanded[['수업구성요소']].explode('수업구성요소')
course_dummies = pd.get_dummies(course['수업구성요소'], prefix='수업구성요소')
course_ohe = course_dummies.groupby(course.index).sum()
course_ohe = course_ohe.reindex(df_expanded.index, fill_value=0)

# 성적평가 one-hot
grade = df_expanded[['성적평가']].explode('성적평가')
grade_dummies = pd.get_dummies(grade['성적평가'], prefix='성적평가')
grade_ohe = grade_dummies.groupby(grade.index).sum()
grade_ohe = grade_ohe.reindex(df_expanded.index, fill_value=0)

special_ohe = special_ohe.astype(bool)
course_ohe = course_ohe.astype(bool)
grade_ohe = grade_ohe.astype(bool)

df = pd.concat([df, special_ohe, course_ohe, grade_ohe], axis=1)

In [138]:
df

,인원제한,NeMo,영강,년도,튜토리얼,MOOC,학수번호,학기,학점,개설학과,...,수업구성요소_체험,수업구성요소_퀴즈,수업구성요소_토론,수업구성요소_특강,수업구성요소_포럼,수업구성요소_프로젝트,수업구성요소_협동학습,성적평가_P/F,성적평가_상대평가,성적평가_절대평가
0,True,False,False,2024,False,False,ACEE123,1R,3.0,건축사회환경공학부,...,False,False,False,False,False,False,False,False,False,True
1,True,False,False,2024,False,False,ACEE219,1R,3.0,건축사회환경공학부,...,False,True,False,False,False,False,False,False,False,True
2,True,False,False,2024,False,False,ACEE227,1R,3.0,건축사회환경공학부,...,False,False,False,False,False,False,False,False,False,True
3,True,False,False,2024,False,False,ACEE227,1R,3.0,건축사회환경공학부,...,False,False,False,False,False,False,False,False,False,True
4,True,False,False,2024,False,False,ACEE228,1R,3.0,건축사회환경공학부,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33429,True,False,False,2020,False,False,TEEN303,2R,3.0,기술창업융합전공,...,False,False,False,False,False,False,False,False,False,True
33430,True,False,False,2020,False,False,TEEN306,2R,3.0,기술창업융합전공,...,False,False,True,False,False,False,True,False,False,True
33431,True,False,False,2020,False,False,TEEN312,2R,6.0,기술창업융합전공,...,False,False,False,False,False,False,False,True,False,True
33432,True,False,False,2020,False,False,TEEN312,2R,6.0,기술창업융합전공,...,False,False,False,False,False,False,False,True,False,True


In [159]:
t = pd.read_pickle("평가방식.pickle")

l = []
label = ["시험", "출석", "과제", "참여도", "발표", "프로젝트", "협동", "보고서", "실험", "태도", "기타"]
for r in df["detailed_info"]:
    zero_dict = {key: 0 for key in label}
    for key in r["평가방법"]:
        try:
            if key == "합계": continue
            elif key not in t:
                zero_dict["기타"] += float(re.search(r'\d+', r["평가방법"][key]).group()) * 0.01
                continue
            zero_dict[t[key]] += float(re.search(r'\d+', r["평가방법"][key]).group()) * 0.01
        except:
            continue
    l.append(zero_dict)

In [160]:
l

[{'시험': 0,
  '출석': 0,
  '과제': 0,
  '참여도': 0,
  '발표': 0,
  '프로젝트': 0,
  '협동': 0,
  '보고서': 0,
  '실험': 0,
  '태도': 0,
  '기타': 0},
 {'시험': 0,
  '출석': 0,
  '과제': 0,
  '참여도': 0,
  '발표': 0,
  '프로젝트': 0,
  '협동': 0,
  '보고서': 0,
  '실험': 0,
  '태도': 0,
  '기타': 0},
 {'시험': 0.9,
  '출석': 0.1,
  '과제': 0,
  '참여도': 0,
  '발표': 0,
  '프로젝트': 0,
  '협동': 0,
  '보고서': 0,
  '실험': 0,
  '태도': 0,
  '기타': 0},
 {'시험': 0,
  '출석': 0,
  '과제': 0,
  '참여도': 0,
  '발표': 0,
  '프로젝트': 0,
  '협동': 0,
  '보고서': 0,
  '실험': 0,
  '태도': 0,
  '기타': 0},
 {'시험': 0,
  '출석': 0,
  '과제': 0,
  '참여도': 0,
  '발표': 0,
  '프로젝트': 0,
  '협동': 0,
  '보고서': 0,
  '실험': 0,
  '태도': 0,
  '기타': 0},
 {'시험': 0,
  '출석': 0,
  '과제': 0,
  '참여도': 0,
  '발표': 0,
  '프로젝트': 0,
  '협동': 0,
  '보고서': 0,
  '실험': 0,
  '태도': 0,
  '기타': 0},
 {'시험': 0,
  '출석': 0,
  '과제': 0,
  '참여도': 0,
  '발표': 0,
  '프로젝트': 0,
  '협동': 0,
  '보고서': 0,
  '실험': 0,
  '태도': 0,
  '기타': 0},
 {'시험': 0,
  '출석': 0,
  '과제': 0,
  '참여도': 0,
  '발표': 0,
  '프로젝트': 0,
  '협동': 0,
  '보고서': 0,
  '실험': 0,
  '태도': 0,


In [161]:
df2 = pd.DataFrame(l)

In [163]:
df = pd.concat([df, df2], axis=1)

In [164]:
df

,인원제한,NeMo,영강,년도,튜토리얼,MOOC,학수번호,학기,학점,개설학과,...,출석,과제,참여도,발표,프로젝트,협동,보고서,실험,태도,기타
0,True,False,False,2024,False,False,ACEE123,1R,3.0,건축사회환경공학부,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,True,False,False,2024,False,False,ACEE219,1R,3.0,건축사회환경공학부,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,True,False,False,2024,False,False,ACEE227,1R,3.0,건축사회환경공학부,...,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,True,False,False,2024,False,False,ACEE227,1R,3.0,건축사회환경공학부,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,True,False,False,2024,False,False,ACEE228,1R,3.0,건축사회환경공학부,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33429,True,False,False,2020,False,False,TEEN303,2R,3.0,기술창업융합전공,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33430,True,False,False,2020,False,False,TEEN306,2R,3.0,기술창업융합전공,...,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0
33431,True,False,False,2020,False,False,TEEN312,2R,6.0,기술창업융합전공,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33432,True,False,False,2020,False,False,TEEN312,2R,6.0,기술창업융합전공,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [165]:
df.drop(columns=["detailed_info"])

,인원제한,NeMo,영강,년도,튜토리얼,MOOC,학수번호,학기,학점,개설학과,...,출석,과제,참여도,발표,프로젝트,협동,보고서,실험,태도,기타
0,True,False,False,2024,False,False,ACEE123,1R,3.0,건축사회환경공학부,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,True,False,False,2024,False,False,ACEE219,1R,3.0,건축사회환경공학부,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,True,False,False,2024,False,False,ACEE227,1R,3.0,건축사회환경공학부,...,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,True,False,False,2024,False,False,ACEE227,1R,3.0,건축사회환경공학부,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,True,False,False,2024,False,False,ACEE228,1R,3.0,건축사회환경공학부,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33429,True,False,False,2020,False,False,TEEN303,2R,3.0,기술창업융합전공,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33430,True,False,False,2020,False,False,TEEN306,2R,3.0,기술창업융합전공,...,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0
33431,True,False,False,2020,False,False,TEEN312,2R,6.0,기술창업융합전공,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33432,True,False,False,2020,False,False,TEEN312,2R,6.0,기술창업융합전공,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [167]:
df.to_csv('final.csv', index=False, encoding='utf-8')
